## Context

As a Data Scientist, you work for Hass Consulting Company which is a real estate leader with over 25 years of experience. You have been tasked to study the factors that affect housing prices using the given information on real estate properties that was collected over the past few months. Later onwards, create a model that would allow the company to accurately predict the sale of prices upon being provided with the predictor variables. 

## Experimental design undertaken

## Exploring the dataset

Here we will seek to understand the dataset, check for data types, dimensionality, missing data, correlation etc

In [0]:
# Import necessary libraries

# import libraries
import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sp

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
## --PERSONAL NOTE-- ##
#Newer versions of matplotlib have broken Seaborn. Until it gets fixed, you gotta downgrade son.


!pip install matplotlib==3.1.0

     |████████████████████████████████| 13.1MB 2.8MB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.1.2
    Uninstalling matplotlib-3.1.2:
      Successfully uninstalled matplotlib-3.1.2


In [3]:
# Load the dataset

housedf = pd.read_csv ('house_data.csv')
housedf2 = housedf

housedf.head(5)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


---
**Dataset glossary**

price  - Price of the house

bedrooms - Number of Bedrooms

bathrooms - Number of Bathrooms

sqft_living - Square feet area of living area

sqft_lot  - Square feet area of parking Layout

floors - Number of Floors

waterfront - Whether waterfront is there or not

view - Number of Views

grade - Grades

sqft_basement - Square feet area off basement

yr_built - Year the house is built

yr_renovated - Year the house is renovated

zipcode - zipcode os the house

lat : Latitude of the house

lon : Longitude of the house

---

## Understanding the dataset

In [5]:
# Checking shape, data types

housedf.shape

(21613, 20)

In [6]:
housedf.dtypes

id                 int64
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

It seems odd that the _floors_ and _bathrooms_ columns would be of data type 'float'. However, I know that in real estate, bathrooms can be counted as a half if they only have a toilet with no shower. Not sure what that is for floors, but I'll generate a few random records to explore and get a better idea before proceeding. 

In [7]:
# Generate random columns

housedf.take(np.random.permutation(len(housedf))[:15])

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
5990,1089000190,925000.0,4,2.25,2590,13894,2.0,0,0,4,9,2590,0,1975,0,98005,47.6351,-122.165,2720,13894
3733,6744700900,795000.0,4,2.50,2570,13450,1.0,0,4,3,8,1510,1060,1948,0,98155,47.7429,-122.285,3470,12615
6592,6616000010,814000.0,4,2.50,2840,8820,1.0,0,2,5,8,1420,1420,1952,0,98118,47.5542,-122.265,2310,8750
19960,3821700038,305000.0,3,3.00,1290,1112,3.0,0,0,3,7,1290,0,2008,0,98125,47.7282,-122.296,1230,9000
2567,6071700160,603500.0,6,2.75,2660,8400,1.0,0,0,5,8,1550,1110,1962,0,98006,47.5490,-122.173,2280,8400
8171,795000405,285950.0,2,1.00,1170,6000,1.0,0,0,3,6,1170,0,1948,0,98168,47.5033,-122.331,1130,7500
7331,5101402472,340500.0,2,1.00,940,5413,1.0,0,0,3,7,940,0,1923,0,98115,47.6956,-122.304,1340,5296
17250,1442700430,499950.0,5,2.50,3180,23809,1.0,0,0,3,9,3180,0,1978,0,98038,47.3727,-122.054,2500,15778
5249,8802400415,205000.0,3,1.00,1050,8498,1.0,0,0,3,7,1050,0,1958,0,98031,47.4038,-122.203,1340,8498
6192,3726800285,346000.0,2,1.00,1070,2196,1.0,0,0,4,7,880,190,1917,0,98144,47.5726,-122.308,1160,3600


Upon closer inspection, indeed there are incomplete (.5) floors. After researching, I can see that it's not an error. A property with 1.5 floors has the master bedroom on one level, and all other rooms on another.

In [8]:
# Checking for null values

housedf.isnull().any()

id               False
price            False
bedrooms         False
bathrooms        False
sqft_living      False
sqft_lot         False
floors           False
waterfront       False
view             False
condition        False
grade            False
sqft_above       False
sqft_basement    False
yr_built         False
yr_renovated     False
zipcode          False
lat              False
long             False
sqft_living15    False
sqft_lot15       False
dtype: bool

In [10]:
# Check for duplicate rows

duplicates = housedf[housedf.duplicated()]
duplicates.shape

(0, 20)

There are 3 duplicate rows, which will now get dropped!

In [0]:
# Drop duplicate rows

housedf = housedf.drop_duplicates()

Our dataset is complete. There are no null values. It's important to note that this doesn't mean that there aren't any errors, say from transcription, or other data recording forms. The data formats, column headers etc all seem OK, and I have a much better understanding of the dataset. I can now perform descriptive analysis 

## Exploratory Data Analysis

In [11]:
housedf.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161000e+04,2.161000e+04,21610.000000,21610.000000,21610.000000,2.161000e+04,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000
mean,4.580161e+09,5.401789e+05,3.370847,2.114739,2079.881212,1.510829e+04,1.494239,0.007543,0.234197,3.409440,7.656779,1788.347894,291.533318,1971.003609,84.322351,98077.945673,47.560049,-122.213910,1986.518695,12769.031976
std,2.876547e+09,3.673876e+05,0.930110,0.770204,918.500299,4.142323e+04,0.539994,0.086523,0.766136,0.650764,1.175500,828.138723,442.596699,29.372639,401.499264,53.505373,0.138572,0.140833,685.425781,27305.972464
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.216125e+05,3.000000,1.750000,1425.500000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.470925,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.619000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068875e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


I will use Pandas Profiling to perform univariate and bivariate analysis, and check to see how strongly each variable is correlated to all the others.

In [4]:
import pandas_profiling

pandas_profiling.ProfileReport(housedf)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,20
Number of observations,21613
Total Missing (%),0.0%
Total size in memory,3.3 MiB
Average record size in memory,160.0 B
Numeric,19
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


A few observations:
1. What is immediately apparent is that all measurements of square footage (living space, basement, parking lot) are positively correlated with the price. As one goes up, so does the other.
2. Location, which is ususally a key indicator for property value  in the real estate business, doesn't really help us here since the locations we're given are in latitude + longitude format. One could go to extremes by checkig to see what locations the coordinates translate to, and creating clusters for the same, hoping to find that all the properties listed in the dataset are clustered in similar ways. Only then can one use this categorical data as part of the analysis. 
3. Year built/ renovated don't seem too have much bearing in our dataset as well, unlike with real world examples where one may observe that older houses in certain areas, with a certain je ne sais quoi, may attract higher prices

#### Multivariate Analysis

In [0]:
# preprocessing
X = housedf.drop ('price', 1)
y = housedf ['price']

# splitting the data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Normalization
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Applying PCA
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

##### Checking for correlations (VIF)


In [6]:
corr = housedf.corr()
corr



,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016797,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798
price,-0.016797,1.000000,0.308338,0.525134,0.702044,0.089655,0.256786,0.266331,0.397346,0.036392,0.667463,0.605566,0.323837,0.053982,0.126442,-0.053168,0.306919,0.021571,0.585374,0.082456
bedrooms,0.001286,0.308338,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.005160,0.525134,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,-0.012258,0.702044,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,-0.132109,0.089655,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.018525,0.256786,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,-0.002721,0.266331,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.011592,0.397346,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,-0.023783,0.036392,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406


In [0]:
plt.figure(figsize = (15, 10))
sb.heatmap(X.corr(), annot = True) 
plt.title('Correlation Heatmap')
plt.show()

In [0]:
# This function will calculate VIF and drop highly correlated variables

from statsmodels.stats.outliers_influence import variance_inflation_factor    
def calculate_vif_(X, thresh):
  cols = X.columns
  variables = np.arange(X.shape[1])
  dropped=True
  while dropped:
      dropped=False
      c = X[cols[variables]].values
      vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
      maxloc = vif.index(max(vif))
      if max(vif) > thresh:
          print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
          variables = np.delete(variables, maxloc)
          dropped=True
  print('Remaining variables:')
  print(X.columns[variables])
  return X[cols[variables]]

In [9]:
# X is the 'housedf' dataset, minus the price column, which is what we're trying to predict using our model.
# 4 is the threshold we're setting for VIF

calculate_vif_(X, 4)

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'sqft_living' at index: 3
dropping 'sqft_above' at index: 9
Remaining variables:
Index(['id', 'bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront',
       'view', 'condition', 'grade', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')


,id,bedrooms,bathrooms,sqft_lot,floors,waterfront,view,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,3,1.00,5650,1.0,0,0,3,7,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,3,2.25,7242,2.0,0,0,3,7,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2,1.00,10000,1.0,0,0,3,6,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,4,3.00,5000,1.0,0,0,5,7,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,3,2.00,8080,1.0,0,0,3,8,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,3,2.50,1131,3.0,0,0,3,8,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,4,2.50,5813,2.0,0,0,3,8,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,2,0.75,1350,2.0,0,0,3,7,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,3,2.50,2388,2.0,0,0,3,8,0,2004,0,98027,47.5345,-122.069,1410,1287


LEt's assess the heatmap. Using a threshold of 0.7 fro our heatmap assessment, we can use the heatmap to see which variables are highly correlated:



1.   Bathrooms vs sqft_living
2.   sqft_living vs grade
3. sqft_living vs sqft_above
4. grade vs sqft_above

There's nothing unusual here. Number of bathrooms will go up as square footage goes up.
ALso, assuming the grades are ordinal in nature, it makes sense that the higher grade houses have more square footage. Once again, nothing unusual there.

These results are supported by the output of the custom function  in the cell above the heatmap. It's no surprise that the two features it picked as having VIFs beyond our desired threshold.

All the measures of square footage are tied into almost everything else.
If we choose to remove one variable from each of the pairs listed above, we get the same result as we did with the VIF assessment function.

We can remove the sqft_living and sqft_above features and see how they affect our accuracy, compared to accuracy when we leave them in.




# Modeling
## Multiple Linear Regression

In [0]:
# Subsetting the data
X = housedf[['bedrooms', 'sqft_lot', 'sqft_above', 'condition', 'yr_built','grade','sqft_basement']]
y = housedf['price']

I feel that removing the sqft_living and sqft_above variables is a crazy thing to do, seeing as they're our two most prominent measures of  Square Footage, but the accuracy will tell all.

In [0]:
# Dividing our data into training and test sets
# ---
# 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [12]:
# Training the Algorithm
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
# coefficients for our test set attributes. 
## PERSONAL NOTE ##
# Coefficient = SLope 

coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
bedrooms,-41929.781901
sqft_lot,-0.329274
sqft_above,197.264599
condition,19973.750571
yr_built,-3349.760677
grade,142619.467402
sqft_basement,210.115954


Below are our predictions compared side by side with the actual, expected values

In [0]:
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values (without sqft_living and sqft_above)
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,452347.721441
7573,320000.0,373921.914343
12873,245000.0,442744.430295
209,464000.0,629456.463528
19155,190000.0,12621.214042
...,...,...
11080,368888.0,422043.965175
18910,734200.0,650067.034180
15838,411000.0,702926.979414
8695,410000.0,705791.314014


Oof! The predictions are waaaay off. As stated before, the 2 features that the VIF function recommended for dropping are the key indicators for square footage, so we cannot possibly make predictions without them. This proves that. 

Let's try it with just the _sqft_above_ feature, which has a lower correlation index compared to _sqft_living_. Modifications are made by adding and subtracting features from X and y above.

In [14]:
# This is the version that includes sqft_above ONLY
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
957,323000.0,4.995287e+05
14143,690000.0,6.330798e+05
19452,142000.0,-6.493529e+05
20510,560000.0,3.358024e+05
18354,545000.0,3.550805e+05
...,...,...
5349,386591.0,3.817564e+05
11082,344950.0,5.254779e+05
4413,1150000.0,1.129544e+06
17461,622200.0,4.243678e+05


Our results are much better. We can look at the last version where we keep both features. We will do proper accuracy checks later as well.

In [39]:
# This is the version that includes both sqft_above and sqft_living
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


In [32]:
# This is the version that includes sqft_living alone

# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


Judging just from observation, it seems that we get similar levels of accuracy when both features are included. 

## Evaluating our linear regression model

I will do two versions of these tests, just to see in numbers, the difference between the model when you include one of our variables in question, and when you include both.

In [40]:
# Evaluating the Algorithm, when you include both features
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837732
Mean Squared Error: 57719331207.42597
Root Mean Squared Error: 240248.4780543385


In [33]:
# Evaluating the Algorithm, when you include ONLY sqft_living
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837715
Mean Squared Error: 57719331207.42588
Root Mean Squared Error: 240248.4780543383


In [15]:
# Evaluating the Algorithm, when you include ONLY sqft_above
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148317.1684772434
Mean Squared Error: 57090904671.09287
Root Mean Squared Error: 238937.03076562425


All three versions described above produce the same MAE, MSE And RMSE. 

Going forward I'll use ONLY _sqft_above_.

In [47]:
# R2 score

from sklearn.metrics import r2_score
m = r2_score(y_test, y_pred)
m

0.6102276905307119

In [0]:
# Plotting the residual plot
# Residuals have been calculated by by substracting the test value from the predicted value
# 

residuals = np.subtract(y_pred, y_test)

# Plotting the residual scatterplot
#
plt.scatter(y_pred, residuals, color='black')
plt.title('Residual Plot')
plt.ylabel('residual')
plt.xlabel('fitted values')
plt.axhline(y = float(residuals.mean()), color='red', linewidth=1)
plt.show()


In [19]:

# Barlett's test for heteroskedasticity.


import scipy as sp
test_result, p_value = sp.stats.bartlett(y_pred, residuals)
# To interpret the results we must also compute a critical value of the chi squared distribution

degree_of_freedom = len(y_pred)-1
probability = 1 - p_value
critical_value = sp.stats.chi2.ppf(probability, degree_of_freedom)
print(critical_value)

# If the test_result is greater than the critical value, then we reject our null
# hypothesis. This would mean that there are patterns to the variance of the data
# Otherwise, we can identify no patterns, and we accept the null hypothesis that 
# the variance is homogeneous across our data
if (test_result > critical_value):
  print('the variances are unequal, and the model should be reassessed')
else:
  print('the variances are homogeneous!')

inf
the variances are homogeneous!


# Quantile Regression

In [21]:
import statsmodels.formula.api as smf

# Dividing our data into training and test sets
# ---
# 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Finding the regression coefficients for the conditioned median, 0.5 quantile
#
mod = smf.quantreg('y ~ X', housedf)
res = mod.fit(q=.5)

# Then print out the summary of our model
#
print(res.summary())

                         QuantReg Regression Results                          
Dep. Variable:                      y   Pseudo R-squared:               0.3647
Model:                       QuantReg   Bandwidth:                   2.524e+04
Method:                 Least Squares   Sparsity:                    3.483e+05
Date:                Sun, 02 Feb 2020   No. Observations:                21613
Time:                        19:32:40   Df Residuals:                    21605
                                        Df Model:                            7
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.756e+06   9.56e+04     28.820      0.000    2.57e+06    2.94e+06
X[0]       -2.756e+04   1588.628    -17.348      0.000   -3.07e+04   -2.44e+04
X[1]          -0.0937      0.029     -3.201      0.001      -0.151      -0.036
X[2]         132.2671      2.589     51.091      0.0

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)


#### Making predictions

In [22]:
 
y_pred = regressor.predict(x_test)

# To compare the actual output values for X_test with the predicted values
# 
quan = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
quan

,Actual,Predicted
957,323000.0,4.995287e+05
14143,690000.0,6.330798e+05
19452,142000.0,-6.493529e+05
20510,560000.0,3.358024e+05
18354,545000.0,3.550805e+05
...,...,...
5349,386591.0,3.817564e+05
11082,344950.0,5.254779e+05
4413,1150000.0,1.129544e+06
17461,622200.0,4.243678e+05


#### Evaluating the Algorithm


In [23]:

# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148317.1684772434
Mean Squared Error: 57090904671.09287
Root Mean Squared Error: 238937.03076562425


In [24]:
#R2 score
from sklearn.metrics import r2_score
q = r2_score(y_test, y_pred)
q

0.6096865094006017

Our accuracy is just about the same as with the linear model.

60% vs 61%

In [0]:
# Plotting the residual plot
# Residuals have been calculated by by substracting the test value from the predicted value
# 

residuals = np.subtract(y_pred, y_test)
# Plotting the residual scatterplot
#
plt.scatter(y_pred, residuals, color='black')
plt.title('Residual Plot')
plt.ylabel('residual')
plt.xlabel('fitted values')
plt.axhline(y = float(residuals.mean()), color='red', linewidth=1)
plt.show()

# Ridge Regression

In [0]:

# Importing our libraries
# 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [0]:

# Setting predictors and predicted features
#
x = housedf[['bedrooms', 'bathrooms', 'sqft_above', 'sqft_lot', 'floors','condition', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15',
'sqft_lot15']]
y = housedf['price']

In [0]:
# Splitting the dataset into training and testing sets
#
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 10)
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [30]:
# Creating our baseline regression model
# This is a model that has no regularization
# 
regression = LinearRegression()
regression.fit(x_train,y_train)
y_pred=regression.predict(x_test)
first_model = (mean_squared_error(y_test, y_pred))
# first_model = (mean_squared_error(y_true=y,y_pred=regression.predict(x_test)))
print(first_model)

62903720909.19179


In [0]:
# determining the most appropriate value for the l2 regularization.
 
ridge = Ridge(normalize=True)
search = GridSearchCV(estimator=ridge,param_grid={'alpha':np.logspace(-5,2,8)},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [32]:
# We now use the .fit function to run the model and then use the .best_params_ and
#  .best_scores_ function to determine the models strength. 
# 
search.fit(x_train,y_train)
search.best_params_

{'alpha': 0.001}

In [33]:
search.best_score_

-57942891435.690384

In [34]:
# We can confirm this by fitting our model with the ridge information and finding the mean squared error below
#
ridge = Ridge(normalize=True,alpha=0.001)
ridge.fit(x_train,y_train)
second_model = (mean_squared_error(y_true=y_test,y_pred=ridge.predict(x_test)))
print(second_model)

62909465959.67396


In [35]:
# Making predictions
#
y_pred = ridge.predict(x_test)
y_pred
# To compare the actual output values for P_test with the predicted values
# 
r = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
r

,Actual,Predicted
957,323000.0,5.085278e+05
14143,690000.0,5.376131e+05
19452,142000.0,-5.099233e+05
20510,560000.0,4.025400e+05
18354,545000.0,2.616187e+05
...,...,...
6939,800000.0,7.342564e+05
19910,430000.0,4.225951e+05
20466,750000.0,7.696969e+05
14961,1480000.0,1.156522e+06


In [36]:
# CHecking accuracy using R2 score


from sklearn.metrics import r2_score
r = r2_score(y_test, y_pred)
r

0.5530780215725002

# Conclusion

Using the models above, we had our accuracy fluctuating between 55% and 61%, with Ridge regression scoring lowest. It is worth noting though, that we included _sqft_living15_ and _sqft_living15_ which weren't present in the other models. This is a classic example of how increasing complexity of the dataset (more features) may not necessarily yield a better accuracy.

This accuracy can be improved by scrutinizing our features more, and deciding what to leave out.

ALso, if we were to assign weights to the different features, anything to do with sq. footage had too much sway compared to the others.